# MC2-Project-1 Build a Market Simulator

In [127]:
# http://quantsoftware.gatech.edu/MC2-Project-1

from DataHelper import DataHelper

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.optimize as spo

In [128]:
# input: orders file (name of file), initial cash
# output: history of value

#  *** ASSUMPTIONS ***
# if we sell a symbol that we don't own, then we are shorting
# orders are filled with adj. close prices

# psuedo algorithm to process orders file:

# step 1 - create Prices dataFrame 
#    read in data file, what's earliest date, last date, call get data
#    make dataframe with date, [stock symbols], cash
#    cash is equal to 1.0 for each day (when dealing with foreign currencies could be other value)

# step 2 - create a trades dataFrame to log change of shares in stocks
#    date, [stock symbols that holds volume], cash (start at 0)
#    initialize with 0 and populate step by step 

# step 3 - populate trades dataFrame
#    step through orders file line by line and process (is just logging, doesn't total items)
#    will need for loop

# step 4 - create a holdings dataFrame
#    date, [stock symbols], cash
#    initialize everything to 0 and 1st day with initial cash
#    loop through trades dataFrame and populate each day with any trades + previous day volume

# step 5 - create a Values dataFrame
#    represents the value of dollars of each asset we hold
#    Values = holdings * price dataFrame
#
# step 6 - sum (axis=1) of Values of dataFrame to get daily value;


### Leverage

### ${ Leverage = \frac{\sum |investements|}{\sum investments  + cash} }$

sum (abs (investments)) / sum(investments) + cash


In [129]:
# e.g.

#Date,Symbol,Order,Shares
#2011-01-05,AAPL,BUY,1500
#2011-01-20,AAPL,SELL,1500

# daily value

#2011-01-05     997495.775
#2011-01-06     997090.775
#2011-01-07    1000660.775
#2011-01-10    1010125.775
#2011-01-11    1008910.775
#2011-01-12    1013065.775
#2011-01-13    1014940.775
#2011-01-14    1019125.775
#2011-01-18    1007425.775
#2011-01-19    1004725.775
#2011-01-20     993036.375
start_balance = 1000000

In [130]:
def getOrdersDataFrame(fileName):
    #'testcases_mc2p1/orders-short.csv'
    df = pd.read_csv(fileName, index_col='Date', parse_dates=True, na_values=['nan'])
    return df.sort_index()

def getPricesDataFrame(ordersData):
    df = DataHelper.GetPriceHistory(ordersData['Symbol'].unique().tolist(), ordersData.index.min(), ordersData.index.max(), False)
    df['USDCASH'] = 1.0
    return df

def getTradesDataFrame(ordersData1, pricesData):
    df = pricesData.copy()
    #df['TxFees'] = 0.0
    df[:] = 0.0
    for index, row in ordersData1.iterrows():
        symbol = row['Symbol']
        order = row['Order']
        shares = row['Shares']
        price = pricesData.ix[index][symbol]
        fees = 9.95 + price * .005 * shares

        if(order == 'BUY'):
            df.ix[index][symbol] = shares
        else:
            df.ix[index][symbol] = -shares

        df.ix[index]['USDCASH'] = df.ix[index]['USDCASH'] + (-df.ix[index][symbol] * price) - fees
        #df.ix[index]['TxFees'] = df.ix[index]['TxFees'] + fees
    return df

def getHoldingsDataFrame(tradeData):
    holdings = tradeData.copy()
    holdings[:] = 0.0

    holdings.ix[0]['USDCASH'] = start_balance

    prevIndex = -1
    for index, row in tradeData.iterrows():
        if(prevIndex == -1):
            prevIndex = index

        holdings.ix[index] = tradeData.ix[index] + holdings.ix[prevIndex]
        prevIndex = index


    return holdings

def compute_portvals(orders_file, start_val  = 1000000):
    ordersData1 = getOrdersDataFrame(orders_file)
    priceData1 = getPricesDataFrame(ordersData1)
    tradeData1 = getTradesDataFrame(ordersData1, priceData1)
    holdingsData1 = getHoldingsDataFrame(tradeData1)
    valuesData1 = holdingsData1 * priceData1
    return valuesData1.sum(axis=1)

def assess_fund(dailyValues, rfr=0.0, sf=252.0):
    normed = DataHelper.normalize_data(dailyValues)
    cum_ret = (normed[-1] / normed[0]) - 1
    daily_ret = DataHelper.compute_daily_returns(normed)[1:,]
    mean = daily_ret.mean()
    std = daily_ret.std()
    sharpe_ratio = ((mean - rfr) / std) * (sf ** (1/2.0))
    stats = {"cum_ret": cum_ret, "daily_ret": mean, "std_ret": std, "sharpe_ratio": sharpe_ratio}
    return stats
    

In [132]:
ordersData = getOrdersDataFrame('testcases_mc2p1/orders-short.csv')
ordersData

,Symbol,Order,Shares
Date,,,
2011-01-05,AAPL,BUY,1500
2011-01-20,AAPL,SELL,1500


In [133]:
priceData = getPricesDataFrame(ordersData)
tradeData = getTradesDataFrame(ordersData, priceData)
holdingsData = getHoldingsDataFrame(tradeData)
valuesData = holdingsData * priceData

In [134]:
holdingsData

,AAPL,USDCASH
2011-01-05,1500.0,498640.775
2011-01-06,1500.0,498640.775
2011-01-07,1500.0,498640.775
2011-01-10,1500.0,498640.775
2011-01-11,1500.0,498640.775
2011-01-12,1500.0,498640.775
2011-01-13,1500.0,498640.775
2011-01-14,1500.0,498640.775
2011-01-18,1500.0,498640.775
2011-01-19,1500.0,498640.775


In [135]:
valuesData

,AAPL,USDCASH
2011-01-05,498855.0,498640.775
2011-01-06,498450.0,498640.775
2011-01-07,502020.0,498640.775
2011-01-10,511485.0,498640.775
2011-01-11,510270.0,498640.775
2011-01-12,514425.0,498640.775
2011-01-13,516300.0,498640.775
2011-01-14,520485.0,498640.775
2011-01-18,508785.0,498640.775
2011-01-19,506085.0,498640.775


In [136]:
valuesData.sum(axis=1)

2011-01-05     997495.775
2011-01-06     997090.775
2011-01-07    1000660.775
2011-01-10    1010125.775
2011-01-11    1008910.775
2011-01-12    1013065.775
2011-01-13    1014940.775
2011-01-14    1019125.775
2011-01-18    1007425.775
2011-01-19    1004725.775
2011-01-20     993036.375
dtype: float64

In [137]:
dailyData = compute_portvals('testcases_mc2p1/orders-short.csv')

In [138]:
normed = DataHelper.normalize_data(dailyData)
cum_ret = (normed[-1] / normed[0]) - 1
print(cum_ret)

-0.00447059537671


In [139]:
DataHelper.assess_portfolio('2011-01-10', '2011-12-20', {'$SPX': 1.0})

{'cum_ret': -0.022405985430202779,
 'daily_ret': 1.7295909533977115e-05,
 'sharpe_ratio': 0.018338980744342818,
 'std_ret': 0.01497160915216862}

In [140]:
dailyData = compute_portvals('testcases_mc2p1/orders-short.csv')
assess_fund(dailyData)


{'cum_ret': -0.0044705953767072426,
 'daily_ret': -0.0004272101933083916,
 'sharpe_ratio': -1.0001502536285793,
 'std_ret': 0.006780732744575706}